# Series Finder  

## Find ordered samples based on real-valued variables in the metadata

__Import dependencies and load data__

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%bash
wget https://cran.r-project.org/src/contrib/rjson_0.2.20.tar.gz
R CMD INSTALL rjson_0.2.20.tar.gz

In [ ]:
%%R
library(rjson)

In [ ]:
%%R
metadata_file_tsv <- read.table(file = "./data/experiment_to_terms.tsv", header = FALSE, sep = "\t")

In [ ]:
import json
import pandas as pd
from functions import *

__1. Do you have a set of samples that you would like to restrict the retrieval to?__ 

These may be SRA samples that you have preprocessed and/or have access to expressiond data. If so, point `available_data_f` to a JSON file containing a list of SRA experiment accessions. See `./data/experiments_in_hackathon_data.json` for an example.

In [ ]:
available_data_f = None  ## <-- INPUT HERE

r = load_metadata(available_data_f)
sample_to_terms = r[0]
term_name_to_id = r[1]
sample_to_type = r[2]
sample_to_study = r[3]
sample_to_runs = r[4]
sample_to_real_val = r[5]

__2. Enter your query__  

Enter your target term in place of `'brain'`, your target property in place of `'age'`, and your target unit in place of `None`.

(Note: most samples in the SRA do not have unit information. We advise leaving this as `None` for properties in which the unit is implied (e.g. age is usually expressed in years).

In [ ]:
term = 'brain' ## <-- INPUT HERE
target_property = 'age' ## <-- INPUT HERE
target_unit = 'year' ## <-- INPUT HERE

__3. List terms below to remove__  

In the example below, `'disease', 'disease of cellular proliferation'` will be removed from all timepoints

In [ ]:
blacklist_terms = set([
    'disease', 
    'disease of cellular proliferation'
]) ## <-- INPUT HERE

__4. Search for ordered samples__

In [ ]:
val_to_samples, primary_df = series(term, target_property, sample_to_real_val, sample_to_terms,             
        sample_to_type, sample_to_study, term_name_to_id, blacklist_terms, 
        filter_poor=False, filter_cell_line=True, filter_differentiated=True,
        value_limit=100, target_unit=None)

These are time points that were found:

In [ ]:
create_series_plots(val_to_samples, target_property)
plt.show()

__5. Browse other metadata terms that are associated with samples in a given time point__

Enter whether you want to view cases or controls. Assign the following variable to the number corresponding to the timepoint you would like to view:

In [ ]:
view_value = 38 ## <-- INPUT HERE

if view_value in val_to_samples:
    samples = list(val_to_samples[view_value])
with open('./data/term-in.json', 'w') as f:
    json.dump(samples, f)
print("Displaying data for %d sample with %s=%d" % (len(samples), target_property, view_value))

The following plots the proportion of metadata terms for those terms that appear in at least 10% of the samples in the current subset:

In [ ]:
%%R
source("./Metadata_plot.R")
bp

In [ ]:
%%R
source("./Metadata_table.R")

In [ ]:
%%R
source("./Metadata_piecharts.R")

__6. Produce output file__. 

Enter the filename for which you would like to output these samples:

In [ ]:
output_file = 'series_data.csv' ## <- OUTPUT FILE HERE

primary_df.to_csv(output_file)